<a href="https://colab.research.google.com/github/smraji/AIResidency/blob/main/crewai_multiagent_2025_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guida

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [ ]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "48d3c6e92e2266c9406771edad36b21b7a717130" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=7
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Gemini 3"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Gemini 3.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to start by researching the latest trends, key players, and noteworthy news on Gemini 3.

Action: Search the internet
Action Input: {"search_query": "Gemini 3 latest trends and news"} 


Search results: Title: A new era of intelligence with Gemini 3 - Google Blog
Link: https://blog.google/products/gemini/gemini-3/
Snippet: Gemini 3 Pro outperforms previous models in reasoning, multimodality, and coding benchmarks. Gemini 3 Deep Think mode pushes the boundaries of ...
---
Title: Google's Gemini 3 advances shake up AI competitive landscape
Link: https://finance.ya

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Power of Gemini 3: A Deep Dive into the Future of AI

## Introduction
Gemini 3 is a groundbreaking advancement in the field of artificial intelligence, revolutionizing the way we interact with technology. With its cutting-edge features and recent advancements, Gemini 3 has established itself as a key player in the AI landscape. Its impact on various industries is undeniable, reshaping the way businesses operate and paving the way for a more intelligent future.

## Latest Trends and Key Players
Gemini 3 has shaken up the AI competitive landscape, with key players such as Google, Broadcom, Microsoft, Figma Inc, SAP SE, and Wayfair vying for dominance. The model's unparalleled capabilities have solidified its position as a leader in various sectors, setting new benchmarks for innovation and performance.

## Noteworthy News on Gemini 3
The significant achievements and milestones of Gemini 3 highlight its exceptional capabilities and potential for growth. Key announcements and updates related to Gemini 3 showcase its continuous evolution and commitment to excellence. The model's performance on critical tests and benchmarks further demonstrate its reliability and effectiveness in real-world applications.

## Target Audience Analysis
Marketers, product managers, sales professionals, and AI enthusiasts form the core of Gemini 3's target audience. Marketers can leverage Gemini 3 to revolutionize their strategies, while product managers can enhance their product development processes. Sales professionals will benefit from a deeper understanding of customer interactions, and AI enthusiasts will appreciate the technical intricacies of the model.

## SEO Keywords and Strategies
Integration of SEO keywords such as Google Gemini 3, AI advancements, SEO strategies, keyword research, and AI content optimization is crucial for enhancing visibility and reach. By implementing SEO best practices, content can be optimized for search engines, ensuring that it reaches the target audience effectively.

## Resources and Data
Utilization of relevant resources and data to support claims and insights about Gemini 3 is essential for providing credibility and depth to the content. Data points, case studies, and research findings offer valuable insights into the model's capabilities and potential. Reference to authoritative sources further enhances the reliability of the information presented.

## Call to Action
Readers are encouraged to explore Gemini 3 further, engage with the content, and share insights with others. Feedback, comments, and questions from the audience are welcomed, fostering a community of learning and discussion around the future of AI.

In conclusion, Gemini 3 represents a new era of intelligence, pushing the boundaries of what is possible with AI. Its impact on industries and key players in the field is undeniable, setting new standards for innovation and performance. With a focus on engaging the target audience and optimizing content for search engines, this blog post aims to provide valuable insights and drive discussion around the power of Gemini 3 in shaping the future of AI.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Hypersonic missiles"
result = crew.kickoff(inputs={"topic": topic})
## Key Players in AI Interpretability


 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Hypersonic missiles.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather the latest trends, key players, and noteworthy news on Hypersonic missiles to prioritize them in the content plan.

Action: Search the internet
Action Input: {"search_query": "latest trends in Hypersonic missiles"} 


Search results: Title: A vision for US hypersonic weapons - Atlantic Council
Link: https://www.atlanticcouncil.org/in-depth-research-reports/issue-brief/a-vision-for-us-hypersonic-weapons/
Snippet: Hypersonic weapons build on these trends. Advanced engine technology and improved materials enable missiles to travel at hypersonic

In [ ]:
from IPython.display import Markdown
Markdown(result)

# The Race to Hypersonic Missiles: Trends, Players, and Implications

## Introduction

Hypersonic missiles have become a focal point in modern warfare due to their incredible speed and maneuverability, posing significant challenges to existing defense systems. The global interest and investment in hypersonic missile technology have been steadily increasing as countries race to develop and deploy these advanced weapons. In this blog post, we will explore the latest trends in hypersonic missiles, delve into the key players in the hypersonic missiles race, discuss noteworthy news and developments in this field, and examine the implications of hypersonic missiles on global security.

## Latest Trends in Hypersonic Missiles

The current state of hypersonic missile development worldwide is characterized by rapid advancements in technology and a growing emphasis on speed and precision. Emerging technologies, such as scramjet engines and advanced guidance systems, are driving the growth of hypersonic missiles, making them more capable and versatile than ever before. The implications of hypersonic missiles in defense strategies are profound, as these weapons have the potential to revolutionize the way conflicts are fought and won.

## Key Players in the Hypersonic Missiles Race

Leading countries such as the US, Russia, and China are investing heavily in hypersonic missile technology, seeking to maintain their strategic advantage in an increasingly complex security environment. Key defense contractors and research institutions play a crucial role in the development and deployment of hypersonic missiles, pushing the boundaries of innovation and technological superiority. A comparison of different countries' approaches to hypersonic missile deployment reveals the diverse strategies and priorities shaping the global hypersonic arms race.

## Noteworthy News and Developments

Recent news articles and reports highlight the ongoing testing and deployment of hypersonic missiles by various countries, raising concerns about the geopolitical implications of these advanced weapons. The challenges and opportunities presented by hypersonic missile technology underscore the need for a comprehensive understanding of the risks and benefits associated with their proliferation. As global security dynamics evolve, the strategic implications of hypersonic missiles on regional and international stability cannot be ignored.

## Call to Action

As readers, staying informed on the latest developments in hypersonic missiles is imperative to grasp the full scope of their impact on defense innovation and strategic deterrence. Engaging in discussions about the ethical and strategic implications of hypersonic missile technology can foster a deeper understanding of the complexities surrounding these advanced weapons. For further reading and exploration of the topic, resources are provided to delve deeper into the world of hypersonic missiles and their implications.

In conclusion, the race to hypersonic missiles represents a critical juncture in modern warfare, where speed, precision, and technological prowess converge to shape the future of global security. By staying informed and engaged in discussions about the implications of hypersonic missile technology, we can navigate the complexities of this evolving landscape and strive for a more secure and stable world.

In [ ]:
Markdown(result)

# Deploy this crew in gradio

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.20.0 which is incompatible.


In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS AI Residency Free Masterclass",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e04656fd1bb368833d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
